This notebook is not used operationally or for any validation, its only purpose is to have a clear understanding of the core functions of the AA workflow. The outputs and dimensions of each main step can thus be identified here.

**Import required libraries and functions**

In [1]:
%cd ..

c:\Users\amine.barkaoui\OneDrive - World Food Programme\Documents\GitHub\anticipatory-action


In [2]:
import os
import datetime
import pandas as pd

from config.params import Params

from AA.helper_fns import (
    read_forecasts,
    read_observations,
    aggregate_by_district,
    merge_un_biased_probs,
    merge_probabilities_triggers_dashboard,
)

from hip.analysis.analyses.drought import (
    get_accumulation_periods,
    run_accumulation_index,
    run_gamma_standardization,
    run_bias_correction,
    compute_probabilities,
)

from hip.analysis.aoi.analysis_area import AnalysisArea

**Define parameters**

The `config/{country}_config.yaml` file gathers all the parameters used in the operational script and that can be customized. For example, the *monitoring_year*, the list of districts or the intensity levels can be defined in that file.

In [3]:
params = Params(iso='ZWE', issue=5, index='SPI')

**Read shapefile**

In [4]:
# Define aoi to read datasets using hip-analysis
area = AnalysisArea.from_admin_boundaries(
    iso3=params.iso.upper(),
    admin_level=2,
    resolution=0.25,
    datetime_range=f"1981-01-01/{params.monitoring_year + 1}-06-30",
)

# Read the shapefile
gdf = area.get_dataset([area.BASE_AREA_DATASET])
gdf

,geometry,Code,Name,adm1_Code,adm0_Code
Name,,,,,
Bulawayo,"POLYGON ((28.6712 -20.0163, 28.6072 -19.9615, ...",1010745,Bulawayo,900969,271
Chitungwiza,"POLYGON ((31 -17.9994, 31.0626 -18.0506, 31.11...",1010758,Chitungwiza,900970,271
Epworth,"POLYGON ((31.2031 -17.8863, 31.1303 -17.8654, ...",1010760,Epworth,900970,271
Harare,"POLYGON ((31.2031 -17.8863, 31.2195 -17.8512, ...",1010771,Harare,900970,271
Harare Rural,"POLYGON ((31.1361 -17.9289, 31.125 -17.888, 31...",1010772,Harare Rural,900970,271
...,...,...,...,...,...
Redcliff,"POLYGON ((29.8419 -19.025, 29.8195 -19.0142, 2...",1010810,Redcliff,900978,271
Shurugwi,"POLYGON ((30.4711 -19.8304, 30.4674 -19.8254, ...",1010817,Shurugwi,900978,271
Shurugwi Town,"POLYGON ((30.0317 -19.6013, 29.9984 -19.607, 3...",1010818,Shurugwi Town,900978,271


**Read forecasts**

In [5]:
# When update is set to False, the downscaled dataset is read from a local folder or a s3 bucket. Otherwise, it is directly read from HDC.
forecasts = read_forecasts(
    area,
    params.issue,
    f"{params.data_path}/data/{params.iso}/zarr/2022/{str(params.issue).zfill(2)}/forecasts.zarr",
    update=False,  # True,
)
forecasts

<xarray.DataArray 'tp' (time: 9416, ensemble: 51, latitude: 28, longitude: 33)> Size: 2GB
dask.array<open_dataset-tp, shape=(9416, 51, 28, 33), dtype=float32, chunksize=(9416, 1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    issue        (time) <U7 264kB dask.array<chunksize=(9416,), meta=np.ndarray>
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 75kB 1981-05-01 1981-05-02 ... 2024-11-30
Attributes:
    nodata:   nan
    units:    mm

**Read observations**

In [6]:
# Observations data reading (already stored as the dataset used is the same as the one used in the pre-season/analytical script)
observations = read_observations(
    area,
    f"{params.data_path}/data/{params.iso}/zarr/{params.calibration_year}/obs/observations.zarr",
)
observations

<xarray.DataArray 'band' (time: 15156, latitude: 28, longitude: 33)> Size: 112MB
dask.array<open_dataset-band, shape=(15156, 28, 33), dtype=float64, chunksize=(1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 121kB 1981-01-01 ... 2022-06-30
Attributes:
    nodata:   nan

**Read pre-computed triggers**

Now that we got all the data we need, let's read the triggers file so we can merge the probabilities with it once we have them.

In [7]:
# Read triggers file
if os.path.exists(f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv"):
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv",
    )
else:
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/triggers/triggers.spi.dryspell.{params.calibration_year}.pilots.csv",
    )
triggers_df

,district,index,category,window,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set
0,Beitbridge,DRYSPELL JF,Normal,Window 2,7.0,8.0,0.14,0.30,NRT,NaN,NaN
1,Beitbridge,SPI DJF,Normal,Window 2,7.0,8.0,0.38,0.00,NRT,NaN,NaN
2,Beitbridge,SPI FM,Normal,Window 2,9.0,10.0,0.31,0.35,NRT,NaN,NaN
3,Beitbridge,SPI JFM,Normal,Window 2,10.0,11.0,0.00,0.37,NRT,NaN,NaN
4,Beitbridge,SPI ND,Normal,Window 1,6.0,7.0,0.35,0.11,NRT,0.24,NaN
...,...,...,...,...,...,...,...,...,...,...,...
73,Rushinga,SPI DJ,Normal,Window 2,9.0,10.0,0.23,0.37,NRT,NaN,NaN
74,Rushinga,SPI FM,Normal,Window 2,11.0,12.0,0.34,0.27,NRT,NaN,NaN
75,Rushinga,SPI JF,Normal,Window 2,8.0,9.0,0.08,0.34,NRT,NaN,NaN
76,Rushinga,SPI ND,Normal,Window 1,6.0,7.0,0.30,0.26,NRT,0.29,NaN


**Get accumulation periods covered by the forecasts of the defined issue month**

In [8]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts,
    params.start_season,
    params.end_season,
    params.min_index_period,
    params.max_index_period,
)
accumulation_periods

{'MJ': (5, 6), 'ON': (10, 11)}

Here we focus on the pipeline for one indicator (one period) so we select a single element from the above dictionary (November-December using October forecasts).

In [9]:
# Get single use case
period_name, period_months = list(accumulation_periods.items())[1] # [4]
period_name, period_months

('ON', (10, 11))

**Run accumulation (sum for SPI)**

In [10]:
# Remove 1980 season to harmonize observations between different indexes 
if int(params.issue) >= params.end_season:
    observations = observations.where(
        observations.time.dt.date >= datetime.date(1981, 10, 1), drop=True
    )

In [11]:
# Accumulation
accumulation_fc = run_accumulation_index(
    forecasts.chunk(dict(time=-1)), params.aggregate, period_months, forecasts=True
)
accumulation_obs = run_accumulation_index(
    observations.chunk(dict(time=-1)), params.aggregate, period_months
)

In [12]:
accumulation_fc

<xarray.DataArray 'tp' (time: 44, ensemble: 51, latitude: 28, longitude: 33)> Size: 8MB
dask.array<where, shape=(44, 51, 28, 33), dtype=float32, chunksize=(1, 1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 352B 1981-01-01 1982-01-01 ... 2024-01-01
Attributes:
    nodata:   nan
    units:    mm

In [13]:
accumulation_obs

<xarray.DataArray 'band' (time: 41, latitude: 28, longitude: 33)> Size: 303kB
dask.array<concatenate, shape=(41, 28, 33), dtype=float64, chunksize=(1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 328B 1981-01-01 1982-01-01 ... 2021-01-01
Attributes:
    nodata:   nan

**Run standardization (SPI)**

In [14]:
# Remove inconsistent observations
accumulation_obs = accumulation_obs.sel(
    time=slice(datetime.date(1979, 1, 1), datetime.date(params.monitoring_year - 1, 12, 31))
)

In [15]:
# Anomaly
anomaly_fc = run_gamma_standardization(
    accumulation_fc.load(),
    params.hist_anomaly_start,
    params.hist_anomaly_stop,
    members=True,
)
anomaly_obs = run_gamma_standardization(
    accumulation_obs.load(),
    params.hist_anomaly_start,
    params.hist_anomaly_stop,
)

c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\aa-env\lib\site-packages\hip\analysis\analyses\drought.py:232: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['date', 'ensemble'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'date', 'ensemble'])` before assigning new coordinate values.
  rfh["time"] = [


In [16]:
anomaly_fc

<xarray.DataArray 'tp' (latitude: 28, longitude: 33, time: 1308)> Size: 10MB
array([[[ 7.550e+02,  4.990e+02,  9.140e+02, ...,  1.980e+02,
          8.080e+02,  3.660e+02],
        [ 8.240e+02,  4.900e+02,  7.880e+02, ...,  2.370e+02,
          8.680e+02,  3.750e+02],
        [ 8.690e+02,  5.560e+02,  7.350e+02, ...,  2.270e+02,
          9.000e+02,  3.600e+02],
        ...,
        [ 1.326e+03,  7.310e+02, -3.550e+02, ...,  5.710e+02,
          2.700e+01, -3.120e+02],
        [ 1.358e+03,  7.250e+02, -4.280e+02, ...,  5.670e+02,
          6.300e+01, -3.010e+02],
        [ 1.382e+03,  7.170e+02, -4.930e+02, ...,  5.620e+02,
          1.000e+02, -2.840e+02]],

       [[ 7.420e+02,  5.840e+02,  9.400e+02, ...,  1.010e+02,
          7.850e+02,  4.010e+02],
        [ 8.440e+02,  6.000e+02,  8.050e+02, ...,  1.170e+02,
          8.470e+02,  4.250e+02],
        [ 9.100e+02,  6.650e+02,  7.430e+02, ...,  1.120e+02,
          8.800e+02,  4.210e+02],
...
        [ 1.147e+03,  3.390e+02,  1.470e+02, ...,  2.930e+02,
         -6.150e+02,  1.182e+03],
        [ 1.206e+03,  3.690e+02,  1.040e+02, ...,  3.510e+02,
         -7.840e+02,  1.182e+03],
        [ 1.255e+03,  3.980e+02,  6.300e+01, ...,  4.070e+02,
         -9.520e+02,  1.174e+03]],

       [[ 2.119e+03,  2.590e+02,  1.094e+03, ..., -2.910e+02,
         -5.170e+02,  5.300e+01],
        [ 2.210e+03,  2.910e+02,  1.026e+03, ..., -2.930e+02,
         -4.370e+02,  2.100e+02],
        [ 2.167e+03,  3.320e+02,  1.013e+03, ..., -2.940e+02,
         -3.590e+02,  3.010e+02],
        ...,
        [ 1.176e+03,  4.030e+02,  2.800e+01, ...,  3.790e+02,
         -5.300e+02,  9.900e+02],
        [ 1.188e+03,  4.360e+02, -1.800e+01, ...,  4.390e+02,
         -6.990e+02,  9.940e+02],
        [ 1.191e+03,  4.680e+02, -6.000e+01, ...,  4.970e+02,
         -8.670e+02,  9.910e+02]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 10kB 1981-01-01 1981-01-02 ... 2024-02-20
Attributes:
    nodata:                 nan
    units:                  mm
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-02-20 00:00:00

In [17]:
anomaly_obs

<xarray.DataArray 'band' (latitude: 28, longitude: 33, time: 41)> Size: 303kB
array([[[ -731.,  1313.,   469., ..., -1334.,   421., -1658.],
        [ -560.,  1246.,   129., ..., -1272.,   483., -1756.],
        [ -294.,  1358.,   177., ..., -1191.,   317., -1965.],
        ...,
        [ -427.,  -156., -1055., ...,  1066.,  -711.,  -137.],
        [ -537.,  -162., -1031., ...,  1170.,  -757.,  -273.],
        [ -669.,  -455., -1072., ...,  1177.,  -858.,   -89.]],

       [[ -172.,   978.,   133., ..., -1151.,   390., -1804.],
        [  239.,   775.,  -232., ..., -1077.,   469., -1998.],
        [  517.,   722.,  -310., ..., -1051.,   349., -2090.],
        ...,
        [ -266.,   248., -1331., ...,   995.,  -566.,   -76.],
        [ -259.,  -149.,  -896., ...,  1068.,  -582.,  -439.],
        [ -489.,  -165.,  -950., ...,  1078.,  -518.,  -345.]],

       [[  187.,   728.,   137., ..., -1082.,   253., -1835.],
        [  605.,   580.,  -210., ..., -1025.,   363., -2071.],
        [  860.,   541.,  -237., ..., -1064.,   355., -2264.],
        ...,
...
        ...,
        [ 1809.,  -763.,  -482., ..., -1088.,   -26., -1014.],
        [ 1481.,  -692.,  -143., ...,  -919.,  -171., -1297.],
        [ 1177.,  -695.,    16., ...,  -984.,     4., -1585.]],

       [[ -613.,  1360.,   476., ..., -1300.,  1772.,   -90.],
        [ -634.,  1401.,   241., ..., -1212.,  1680.,   -16.],
        [ -701.,  1490.,   247., ..., -1583.,  1732.,    46.],
        ...,
        [ 1981.,  -767.,  -686., ..., -1517.,   -72.,  -757.],
        [ 1701.,  -648.,  -424., ..., -1577.,  -542., -1080.],
        [ 1367., -1022.,    57., ..., -1405.,  -488., -1474.]],

       [[ -623.,  1146.,   435., ..., -1149.,  1985.,    33.],
        [ -638.,  1241.,   357., ..., -1052.,  2116.,   164.],
        [ -647.,  1396.,   356., ..., -1274.,  2012.,   195.],
        ...,
        [ 2024.,  -928.,  -580., ..., -1491.,   163.,  -785.],
        [ 1961.,  -892.,  -283., ..., -1849.,  -216., -1097.],
        [ 1753.,  -746.,  -306., ..., -1973.,  -411., -1340.]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 328B 1981-01-01 1982-01-01 ... 2021-01-01
Attributes:
    nodata:                 nan
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-01-01 00:00:00

**Run bias correction**

In [18]:
import numpy as np
import pandas as pd
import xarray as xr
from hdc.algo.dekad import Dekad
from numba import jit
from odc.geo.xr import xr_reproject
from xclim import sdba

from hip.analysis.analyses.drought import ENSO_COLD, ENSO_WARM, ENSO_NEUTRAL

In [19]:
def _get_enso_years(year, issue, monitoring_start):
    for years in [ENSO_NEUTRAL, ENSO_WARM, ENSO_COLD]:
        # TODO this is if meant to replicate R outputs but will be removed in another branch
        if issue >= monitoring_start and year in years:
            return np.array(years)
        elif issue < monitoring_start and year in (np.array(years) + 1):
            return np.array(years) + 1


@jit(nopython=True)
def _get_nn(pos, size):
    """
    Returns the nearest neighbors of a position on a 1D lattice.

    Args:
        pos: int, The position for which nearest neighbors are to be found.
        size: int, The size of the 1D lattice.

    Returns:
        list: a list containing the position itself and its nearest neighbors.

    Notes:
        - If the position is at the edge of the lattice, it returns its single neighbor.
        - For all other positions, it returns the position itself and its adjacent neighbors.
    """
    nn = [pos]

    edge_min = 0
    edge_max = size - 1

    if pos == edge_min:
        nn.append(pos + 1)
    elif pos == edge_max:
        nn.insert(0, pos - 1)
    else:
        nn.insert(0, pos - 1)
        nn.append(pos + 1)

    return nn


def _create_time_coord(n: int):
    """
    Create a time coordinate for reformatted obs for bias correction.
    """
    start = datetime.datetime.strptime("01-01-1900", "%d-%m-%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, n)]
    return date_generated


@jit(nopython=True)
def _stack_neighbors_in_time(observations_np, num_neighbors):
    """
    Stack the values of each pixel's neighbors along with the pixel itself in a single dimension.

    Args:
        observations_np: np.ndarray, Array of observations.
        num_neighbors: int, number of nearest neighbors to consider.

    Returns:
        np.ndarray: Reformatted array of observations with neighbors stacked in time dimension.
    """
    lat_size, lon_size, time_size = observations_np.shape

    # Initialize the array to store the reformatted observations
    stacked_observation_array = np.full(
        (lat_size, lon_size, time_size * (num_neighbors + 1)), np.nan
    )

    # Iterate over each latitude and longitude index
    for lat in range(lat_size):
        for lon in range(lon_size):
            # Get the indices of the nearest neighbors
            lat_indices = np.array(_get_nn(lat, lat_size))
            lon_indices = np.array(_get_nn(lon, lon_size))

            # Extract the sub-array corresponding to the nearest neighbors
            pixel_observations = observations_np[lat_indices][:, lon_indices]

            # Assign the stacked values to the appropriate indices in stacked_observation_array
            stacked_values = []
            for t in range(time_size):
                for i, _ in enumerate(lat_indices):
                    for j, _ in enumerate(lon_indices):
                        stacked_values.append(pixel_observations[i, j, t])
            stacked_observation_array[lat, lon, : len(stacked_values)] = stacked_values

    return stacked_observation_array


def _reformat_neighbors(da_observation, num_neighbors):
    """
    Reformat a DataArray of observations by stacking values of each pixel's neighbors in time dimension.

    Args:
        da_observation: xr.DataArray, DataArray containing observations.
        num_neighbors: int, number of nearest neighbors to consider.

    Returns:
        xr.DataArray: Reformatted DataArray with neighbors stacked in time dimension for bias correction.
    """
    stacked_observation_array = _stack_neighbors_in_time(
        da_observation.values, num_neighbors
    )

    reformatted_da_observation = xr.DataArray(
        stacked_observation_array,
        dims=["latitude", "longitude", "time"],
        coords=dict(
            latitude=("latitude", da_observation.latitude.values),
            longitude=("longitude", da_observation.longitude.values),
            time=_create_time_coord(stacked_observation_array.shape[2]),
        ),
        attrs=da_observation.attrs,
    )

    return reformatted_da_observation

In [20]:
def run_bias_correction(
    forecasts: xr.DataArray,
    observations: xr.DataArray,
    start_monitoring: int,
    year: int = 2022,
    issue: int = None,
    nearest_neighbours: int = 0,
    enso: bool = True,
) -> xr.DataArray:
    """
    Calculate bias-corrected rainfall forecasts for year of interest using Empirical Quantile Mapping method

    Args:
        forecasts: xarray.DataArray, rainfall forecasts dataset
        observations: xarray.DataArray, rainfall observations dataset
        end_season: int, end of season month
        year: int, year to bias correct
        issue: int, issue month of forecasts
        nearest_neighbours: int, number of nearest neighbouring pixels to consider in quantile mapping (only for observations)
        enso: bool, default True, if True then only years of the same enso type as the year to bias-correct are selected in the reference time series

    Returns:
        xarray.DataArray: bias-corrected rainfall forecasts for specified year
    """

    # Get TS to bias correct
    year_to_bc = forecasts.where(forecasts.time.dt.year == year, drop=True)

    # Keep historical data
    hist_obs = observations.where(observations.time.dt.year != year, drop=True)
    hist_fc = forecasts.where(
        forecasts.time.dt.year.isin(hist_obs.time.dt.year.values), drop=True
    )

    # Mask ENSO years
    if enso:
        hist_fc = hist_fc.sel(
            time=hist_fc.time.dt.year.isin(_get_enso_years(year, issue, start_monitoring))
        )
        hist_obs = hist_obs.sel(
            time=hist_obs.time.dt.year.isin(_get_enso_years(year, issue, start_monitoring))
        )

    # If nearest_neighbours >0, reformat obs to stack neighbours valuesin time dimension
    if nearest_neighbours > 0:
        hist_obs = _reformat_neighbors(hist_obs, nearest_neighbours)

    # Reformat time coord
    hist_fc = hist_fc.assign_coords({"time": _create_time_coord(hist_fc.time.size)})

    # Create units attr for QM function
    hist_fc.attrs["units"] = ""
    hist_obs.attrs["units"] = ""
    year_to_bc.attrs["units"] = ""

    # Fit QM model
    QM = sdba.EmpiricalQuantileMapping.train(
        hist_obs, hist_fc, nquantiles=np.arange(0, 1.1, 0.1), group="time"
    )

    # Bias correct forecasts of season of interest
    bc = QM.adjust(year_to_bc, interp="linear", extrapolation="constant")

    return bc

In [25]:
# Bias correction
index_bc = run_bias_correction(
    anomaly_fc,
    anomaly_obs,
    start_monitoring=params.start_monitoring,
    year=params.monitoring_year,
    issue=int(params.issue),
    nearest_neighbours=8,
    enso=True,
)
display(index_bc)

<xarray.DataArray 'scen' (longitude: 33, latitude: 28, time: 51)> Size: 377kB
array([[[-9.89010351e+02,  1.01334978e+03,  1.00226457e+03, ...,
          2.59480349e+02,  1.00988565e+03,  4.44623925e+02],
        [-9.67080910e+02,  1.01437827e+03,  8.72260465e+02, ...,
          1.72052111e+02,  9.93380236e+02,  5.00943662e+02],
        [-9.53833693e+02,  1.03596600e+03,  6.39835573e+02, ...,
          2.25762440e+01,  9.79784427e+02,  5.16806903e+02],
        ...,
        [ 1.99658537e+02,  1.30780130e+03,  4.83490320e+02, ...,
          1.18721438e+02, -4.47741683e+01,  2.79191263e+02],
        [ 1.07894615e+02,  1.30528227e+03,  3.21151316e+02, ...,
         -8.38535645e+00, -9.01491924e+01,  1.60032895e+02],
        [ 8.07602230e+01,  1.23921236e+03,  2.42593379e+02, ...,
         -4.60686567e+01, -1.08926589e+02,  1.14529287e+02]],

       [[-9.96886265e+02,  9.71495327e+02,  9.68674596e+02, ...,
          2.75938938e+02,  9.97587086e+02,  4.03479342e+02],
        [-9.65308747e+02,  9.82034185e+02,  8.32754787e+02, ...,
          1.80822689e+02,  9.96242947e+02,  5.09808219e+02],
        [-9.72991444e+02,  1.01827547e+03,  6.81076548e+02, ...,
         -1.90623321e+01,  1.01123146e+03,  6.19191745e+02],
...
        [-1.60543390e+03,  1.05618667e+03,  2.00989339e+02, ...,
          4.52793508e+02, -9.05783117e+02,  1.30171639e+03],
        [-1.66651000e+03,  1.02145192e+03,  1.25153293e+02, ...,
          5.80847805e+02, -7.90801177e+02,  1.20914438e+03],
        [-1.66261252e+03,  1.07907077e+03,  1.15825961e+02, ...,
          6.97576867e+02, -6.28350901e+02,  1.08984223e+03]],

       [[-6.28501632e+02,  1.29637362e+03,  1.86794913e+03, ...,
          3.14525437e+02,  2.39703484e+01, -3.05989362e+02],
        [-5.53985019e+02,  1.27496453e+03,  2.10900000e+03, ...,
          3.03213259e+02, -3.13658537e+01, -2.98126255e+02],
        [-5.28544617e+02,  1.17933478e+03,  2.28200000e+03, ...,
          3.36742700e+02, -1.87671233e+00, -2.79972784e+02],
        ...,
        [-1.58930747e+03,  1.07681655e+03,  4.08696324e+02, ...,
          5.78515457e+02, -1.26084051e+03,  1.28327876e+03],
        [-1.61268200e+03,  1.12426331e+03,  2.90490188e+02, ...,
          6.59981276e+02, -1.01373635e+03,  1.21314248e+03],
        [-1.58555905e+03,  1.19844979e+03,  1.23109325e+02, ...,
          8.32332890e+02, -8.42275716e+02,  1.10183968e+03]]])
Coordinates:
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * time         (time) datetime64[ns] 408B 2024-01-01 2024-01-02 ... 2024-02-20
    spatial_ref  int32 4B 4326
Attributes:
    nodata:                 nan
    units:                  
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-02-20 00:00:00
    history:                [2024-07-26 16:42:23] : Bias-adjusted with Empiri...
    bias_adjustment:        EmpiricalQuantileMapping(group=Grouper(name='time...

**Run probabilities**

In [26]:
# Change dryspell sign as we compare values to a negative threshold to get probabilities
if params.index == "dryspell":
    anomaly_fc *= -1
    index_bc *= -1
    anomaly_obs *= -1

In [27]:
# Probabilities without Bias Correction
probabilities = compute_probabilities(
    anomaly_fc.where(anomaly_fc.time.dt.year == params.monitoring_year, drop=True),
    levels=params.intensity_thresholds,
).round(2)
display(probabilities)

<xarray.DataArray 'tp' (category: 2, latitude: 28, longitude: 33)> Size: 15kB
array([[[0.25, 0.25, 0.24, ..., 0.22, 0.22, 0.22],
        [0.25, 0.25, 0.25, ..., 0.25, 0.22, 0.22],
        [0.25, 0.25, 0.25, ..., 0.24, 0.24, 0.22],
        ...,
        [0.18, 0.18, 0.18, ..., 0.27, 0.29, 0.31],
        [0.14, 0.16, 0.18, ..., 0.25, 0.27, 0.29],
        [0.14, 0.18, 0.18, ..., 0.27, 0.27, 0.29]],

       [[0.37, 0.33, 0.39, ..., 0.35, 0.33, 0.35],
        [0.35, 0.35, 0.35, ..., 0.39, 0.39, 0.39],
        [0.39, 0.37, 0.37, ..., 0.37, 0.39, 0.41],
        ...,
        [0.31, 0.29, 0.33, ..., 0.47, 0.47, 0.49],
        [0.31, 0.29, 0.31, ..., 0.49, 0.51, 0.47],
        [0.29, 0.29, 0.31, ..., 0.45, 0.47, 0.47]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * category     (category) object 16B 'Moderate' 'Normal'

In [28]:
# Probabilities after Bias Correction
probabilities_bc = compute_probabilities(
    index_bc, levels=params.intensity_thresholds
).round(2)
display(probabilities_bc)

<xarray.DataArray 'scen' (category: 2, longitude: 33, latitude: 28)> Size: 15kB
array([[[0.2 , 0.18, 0.18, ..., 0.  , 0.  , 0.  ],
        [0.22, 0.18, 0.18, ..., 0.  , 0.  , 0.  ],
        [0.2 , 0.18, 0.14, ..., 0.  , 0.  , 0.  ],
        ...,
        [0.16, 0.1 , 0.08, ..., 0.29, 0.25, 0.27],
        [0.22, 0.12, 0.1 , ..., 0.29, 0.27, 0.27],
        [0.18, 0.14, 0.08, ..., 0.35, 0.29, 0.27]],

       [[0.27, 0.35, 0.41, ..., 0.1 , 0.12, 0.1 ],
        [0.29, 0.31, 0.39, ..., 0.08, 0.12, 0.1 ],
        [0.35, 0.31, 0.35, ..., 0.06, 0.12, 0.1 ],
        ...,
        [0.35, 0.33, 0.37, ..., 0.43, 0.45, 0.43],
        [0.33, 0.29, 0.31, ..., 0.43, 0.41, 0.45],
        [0.33, 0.31, 0.27, ..., 0.45, 0.39, 0.37]]])
Coordinates:
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
    spatial_ref  int32 4B 4326
  * category     (category) object 16B 'Moderate' 'Normal'

**Admin-2 level aggregation**

In [40]:
# Aggregate by district
probs_district = aggregate_by_district(probabilities, gdf, params)
probs_bc_district = aggregate_by_district(probabilities_bc, gdf, params)

# Build single xarray with merged unbiased/biased probabilities
probs_by_district = merge_un_biased_probs(
    probs_district, probs_bc_district, params, period_name
)
display(probs_by_district)

<xarray.Dataset> Size: 2kB
Dimensions:      (district: 65, category: 2, issue: 1, index: 1)
Coordinates:
  * district     (district) object 520B 'Beitbridge' 'Bikita' ... 'Zvishavane'
  * category     (category) object 16B 'Moderate' 'Normal'
  * issue        (issue) int64 8B 10
  * index        (index) object 8B 'spi ND'
    spatial_ref  int32 4B 0
Data variables:
    prob         (index, district, category, issue) float64 1kB 0.2978 ... 0.45

**Dataframe formatting**

In [41]:
# Merge probabilities with triggers
probs_df, merged_df = merge_probabilities_triggers_dashboard(
    probs_by_district, triggers_df, params, period_name
)

In [42]:
probs_df

,district,category,issue,index,prob,aggregation
0,Beitbridge,Moderate,10,SPI ND,0.30,SPI 2
1,Beitbridge,Normal,10,SPI ND,0.41,SPI 2
2,Bikita,Moderate,10,SPI ND,0.31,SPI 2
3,Bikita,Normal,10,SPI ND,0.43,SPI 2
4,Bindura,Moderate,10,SPI ND,0.33,SPI 2
...,...,...,...,...,...,...
125,Zaka,Normal,10,SPI ND,0.42,SPI 2
126,Zvimba,Moderate,10,SPI ND,0.31,SPI 2
127,Zvimba,Normal,10,SPI ND,0.42,SPI 2
128,Zvishavane,Moderate,10,SPI ND,0.32,SPI 2


In [45]:
merged_df

,district,index,category,window,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set
0,Beitbridge,DRYSPELL JF,Normal,Window 2,7.0,8.0,0.14,0.30,NRT,NaN,NaN
1,Beitbridge,SPI DJF,Normal,Window 2,7.0,8.0,0.38,0.00,NRT,NaN,NaN
2,Beitbridge,SPI FM,Normal,Window 2,9.0,10.0,0.31,0.35,NRT,NaN,NaN
3,Beitbridge,SPI JFM,Normal,Window 2,10.0,11.0,0.00,0.37,NRT,NaN,NaN
4,Beitbridge,SPI ND,Normal,Window 1,6.0,7.0,0.35,0.11,NRT,0.24,NaN
...,...,...,...,...,...,...,...,...,...,...,...
73,Rushinga,SPI DJ,Normal,Window 2,9.0,10.0,0.23,0.37,NRT,NaN,NaN
74,Rushinga,SPI FM,Normal,Window 2,11.0,12.0,0.34,0.27,NRT,NaN,NaN
75,Rushinga,SPI JF,Normal,Window 2,8.0,9.0,0.08,0.34,NRT,NaN,NaN
76,Rushinga,SPI ND,Normal,Window 1,6.0,7.0,0.30,0.26,NRT,0.29,NaN
